In [7]:
import librosa

import os
noise_dir="/home/xinying/xinying/separated/noise"
speech_dir="/home/xinying/xinying/separated/speech"
noise_files=[]
speech_files=[]
for noise in os.listdir(noise_dir):
    noise_files.append(noise)
for speech in os.listdir(speech_dir):
    speech_files.append(speech)
acc=0
ct=0
all_acc=0
all_ct=0
for noise in noise_files:
    ct+=1
    all_ct+=1
    noise_path=os.path.join(noise_dir,noise)
    noise_wav,sr=librosa.load(noise_path, sr=None)
    if noise_wav.std()<0.08:
        acc+=1
        all_acc+=1
print("noise acc:",acc/ct)
acc=0
ct=0
for speech in speech_files:
    ct+=1
    all_ct+=1
    speech_path=os.path.join(speech_dir,speech)
    speech_wav,sr=librosa.load(speech_path, sr=None)
    if speech_wav.std()>0.08:
        acc+=1
        all_acc+=1
print("speechacc:",acc/ct)
print("all:",all_acc/all_ct)

noise acc: 0.7078576225654802
speechacc: 0.8113831899404368
all: 0.76


In [8]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.models import Sequential
from keras.layers import Flatten, Dense

# 设置随机种子以确保可重复性
np.random.seed(42)

# 定义音频文件路径和标签
# noise_files = ['path_to_noise_1.wav', 'path_to_noise_2.wav', ...]
# speech_files = ['path_to_speech_1.wav', 'path_to_speech_2.wav', ...]
labels = [0] * len(noise_files) + [1] * len(speech_files)
noise_dir="/home/xinying/xinying/separated/noise"
speech_dir="/home/xinying/xinying/separated/speech"
waveform, sample_rate = librosa.load(os.path.join(speech_dir, speech_files[0]), sr=None,duration=10)
# 提取音频特征
# 提取音频特征并进行零填充
def extract_features(root, files, max_length=None):
    features = []
    for file in files:
        waveform, sample_rate = librosa.load(os.path.join(root, file), sr=None,duration=10)
        mfcc = librosa.feature.mfcc(waveform, sample_rate)
        
        # 进行零填充
        if max_length is not None and mfcc.shape[1] < max_length:
            pad_width = max_length - mfcc.shape[1]
            mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
        features.append(np.array(mfcc.T))
    
    return np.array(features)

# 提取特征并进行标准化
noise_features = extract_features(noise_dir, noise_files, max_length=10*sample_rate)
speech_features = extract_features(speech_dir, speech_files,max_length=10*sample_rate)

features = np.concatenate((noise_features, speech_features), axis=0)

mean_list = []
std_list = []
for sample in features:
    sample_mean = np.mean(sample, axis=0)
    sample_std = np.std(sample, axis=0)
    mean_list.append(sample_mean)
    std_list.append(sample_std)

# 计算特征的总体均值和标准差
mean = np.mean(mean_list, axis=0).mean()
std = np.mean(std_list, axis=0).mean()

features = (features - mean) / std

# # 划分数据集为训练集和测试集
# X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# # 将数据展平为MLP所需的输入形状
# X_train = X_train.reshape(X_train.shape[0], -1)
# X_test = X_test.reshape(X_test.shape[0], -1)
# y_train = np.array(y_train)
# y_test = np.array(y_test)


# # 构建MLP模型
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# # 编译模型
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# # 训练模型
# model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

/tmp/ipykernel_3457339/3558783011.py:24: FutureWarning: Pass y=[0.00222778 0.00231934 0.00219727 ... 0.         0.         0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_3457339/3558783011.py:24: FutureWarning: Pass y=[0.00061035 0.00067139 0.00091553 ... 0.         0.         0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_3457339/3558783011.py:24: FutureWarning: Pass y=[-0.01663208 -0.0168457  -0.01013184 ...  0.          0.
  0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_3457339/3558783011.py:24: FutureWarning: Pass y=[-0.0017395  -0.00131226 -0.00048828 ...  0.          0.

In [5]:
#MLP
mean_list = []
std_list = []
for sample in features:
    sample_mean = np.mean(sample, axis=0)
    sample_std = np.std(sample, axis=0)
    mean_list.append(sample_mean)
    std_list.append(sample_std)

# 计算特征的总体均值和标准差
mean = np.mean(mean_list, axis=0).mean()
std = np.mean(std_list, axis=0).mean()

features = (features - mean) / std

# 划分数据集为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# 将数据展平为MLP所需的输入形状
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
y_train = np.array(y_train)
y_test = np.array(y_test)


# 构建MLP模型
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
# model_mlp=model
from keras.models import save_model

# 假设model是你要保存的Keras模型
save_model(model, 'mlp4SN.h5')

/home/xinying/.conda/envs/py310/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-31 15:10:30.667710: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 30s 381ms/step - accuracy: 0.6078 - loss: 2.5113 - val_accuracy: 0.6483 - val_loss: 0.7324
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 25s 329ms/step - accuracy: 0.6871 - loss: 0.7103 - val_accuracy: 0.7017 - val_loss: 0.7439
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 345ms/step - accuracy: 0.7377 - loss: 0.6267 - val_accuracy: 0.6500 - val_loss: 0.8967
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 343ms/step - accuracy: 0.7611 - loss: 0.5086 - val_accuracy: 0.6750 - val_loss: 0.7036
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 346ms/step - accuracy: 0.7579 - loss: 0.5131 - val_accuracy: 0.6883 - val_loss: 0.7036
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 340ms/step - accuracy: 0.7697 - loss: 0.4722 - val_accuracy: 0.7017 - val_loss: 0.6999
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 339ms/step - accuracy: 0.7929 - loss: 0.4208 - val_accuracy: 0.6950 - val_loss: 0.7408
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 26s 336ms/step - accuracy: 0.8016 - loss: 0.4030 - val_accu

In [11]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
model_mlp = load_model('/home/xinying/Speaker2/mlp4SN.h5')
predictions = model_mlp.predict(X_test)
binary_labels = (predictions >= 0.5).astype(int)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 80000, 20), dtype=float32). Expected shape (None, 1600000), but input has incompatible shape (32, 80000, 20)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 80000, 20), dtype=float32)
  • training=False
  • mask=None

In [12]:
X_test.shape

(600, 80000, 20)

In [43]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, binary_labels)
print(accuracy)

0.7066666666666667


In [6]:
##CNN
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
# X_train = np.expand_dims(X_train, axis=2)
# X_test = np.expand_dims(X_test, axis=2)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train = X_train.reshape(1,X_train.shape[0], X_train.shape[1],X_train.shape[2])
X_test = X_test.reshape(1,X_test.shape[0], X_test.shape[1],X_test.shape[2])
# X_test = X_test.unsqueeze(0)
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=X_train[0].shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))


loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')
cnn_model=model

/home/xinying/.conda/envs/py310/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-05-31 15:18:56.944563: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 195933044736 exceeds 10% of free system memory.


: 

In [ ]:
save_model(model, 'cnn4SN.h5')

In [14]:
X_train.shape

(2400, 80000, 20)

In [59]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 设置随机种子以确保可重复性
np.random.seed(42)

# 定义音频文件路径和标签
# noise_files = ['path_to_noise_1.wav', 'path_to_noise_2.wav', ...]
# speech_files = ['path_to_speech_1.wav', 'path_to_speech_2.wav', ...]
labels = [0] * len(noise_files) + [1] * len(speech_files)
noise_dir="/home/xinying/xinying/separated/noise"
speech_dir="/home/xinying/xinying/separated/speech"
# 提取音频特征
def extract_features(root,files):
    features = []
    for file in files:
        waveform, sample_rate = librosa.load(os.path.join(root,file), sr=None)
        mfcc = librosa.feature.mfcc(waveform, sample_rate)
        features.append(mfcc.T)
        # print("mfcc:",mfcc.T.shape)
    features=np.array(features)
    return features

# 提取特征并进行标准化
noise_features = extract_features(noise_dir,noise_files)
speech_features = extract_features(speech_dir,speech_files)
features = np.concatenate((noise_features, speech_features), axis=0)
print(features.shape)

/tmp/ipykernel_2968688/2550529730.py:21: FutureWarning: Pass y=[0.00222778 0.00231934 0.00219727 ... 0.         0.         0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_2968688/2550529730.py:21: FutureWarning: Pass y=[0.00061035 0.00067139 0.00091553 ... 0.         0.         0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_2968688/2550529730.py:21: FutureWarning: Pass y=[-0.01663208 -0.0168457  -0.01013184 ...  0.          0.
  0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)
/tmp/ipykernel_2968688/2550529730.py:21: FutureWarning: Pass y=[-0.0017395  -0.00131226 -0.00048828 ...  0.          0.

(3000,)


/tmp/ipykernel_2968688/2550529730.py:21: FutureWarning: Pass y=[-0.03845215 -0.0078125   0.01229858 ...  0.          0.
  0.        ], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)


In [50]:
waveform, sample_rate = librosa.load("/home/xinying/xinying/separated/speech/1-FADG0=origin_SX19_speech.wav", sr=None)
mfcc = librosa.feature.mfcc(waveform, sample_rate)
# features.append(np.array(mfcc.T))

/tmp/ipykernel_2968688/2069985792.py:2: FutureWarning: Pass y=[-9.1552734e-05 -8.2397461e-04 -1.9226074e-03 ... -2.1362305e-04
 -2.7465820e-04 -1.2207031e-04], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)


In [16]:
from sklearn.decomposition import PCA

# 创建PCA对象，指定要保留的目标维度
n_components = 500  # 指定要降低到的目标维度
pca = PCA(n_components=n_components)

# 假设你的数据集名为data，将其转换为二维数组形式
# data的形状为(2400, 1600000)
data_2d = features.reshape(features.shape[0], -1
                           )

# 使用PCA拟合数据
pca.fit(data_2d)

# 对数据进行降维
data_reduced = pca.transform(data_2d)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(data_reduced, labels, test_size=0.2, random_state=42)

# 将数据展平为MLP所需的输入形状
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)
y_train = np.array(y_train)
y_test = np.array(y_test)



# 构建MLP模型
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


/home/xinying/.conda/envs/py310/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6085 - loss: 15.7993 - val_accuracy: 0.6283 - val_loss: 8.7742
Epoch 2/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7704 - loss: 3.3427 - val_accuracy: 0.6333 - val_loss: 6.8794
Epoch 3/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8756 - loss: 1.1957 - val_accuracy: 0.6500 - val_loss: 6.4865
Epoch 4/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9145 - loss: 0.5549 - val_accuracy: 0.6267 - val_loss: 6.6070
Epoch 5/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9584 - loss: 0.2284 - val_accuracy: 0.6650 - val_loss: 6.6082
Epoch 6/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9653 - loss: 0.1795 - val_accuracy: 0.6350 - val_loss: 6.6995
Epoch 7/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9771 - loss: 0.1569 - val_accuracy: 0.6367 - val_loss: 6.2214
Epoch 8/10
75/75 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9904 - loss: 0.0480 - val_accuracy: 0.6450 - val_loss: 6.4875
Ep

In [13]:
from keras.models import load_model
import librosa
import numpy as np
def extract_features(file_pth):
    features = []

    waveform, sample_rate = librosa.load(file_pth, sr=None)
    mfcc = librosa.feature.mfcc(waveform, sample_rate)
    features.append(mfcc.T)
        # print("mfcc:",mfcc.T.shape)
    features=np.array(features)
    return features

# 提取特征并进行标准化
file_pth="/home/xinying/xinying/separated/noise/1-FADG0=origin_SX19_noise0.wav"
features = extract_features(file_pth)
print(features.shape)
model = load_model('/home/xinying/Speaker2/mlp4SN.h5')
predictions = model.predict(features,batch_size=1)
binary_labels = (predictions >= 0.5).astype(int)

/tmp/ipykernel_3457339/3338744938.py:8: FutureWarning: Pass y=[-0.01159668 -0.01461792 -0.0093689  ...  0.00646973  0.00808716
  0.00036621], sr=8000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mfcc = librosa.feature.mfcc(waveform, sample_rate)


(1, 43, 20)


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1, 43, 20), dtype=float32). Expected shape (None, 1600000), but input has incompatible shape (1, 43, 20)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 43, 20), dtype=float32)
  • training=False
  • mask=None